In [1]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Preguntas ajedrez") \
    .getOrCreate()

In [2]:
fichero = "Sept_20_analysis.csv"

In [3]:
df = spark.read.csv(fichero, header=True)

In [4]:
df.head()

2022-02-17 16:11:52,186 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(GAME='11', BlackElo='1143', BlackRatingDiff='6', Date='2020.09.01', ECO='A02', Event='Rated Blitz game', Opening='Bird Opening', Result='0-1', Site='https://lichess.org/hjb8FYm1', Termination='Time forfeit', TimeControl='300+0', UTCTime='00:00:00', WhiteElo='1180', WhiteRatingDiff='-7', Black_elo_category='Low rating', White_elo_category='Low rating', starting_time='300', increment='0', Game_type='Blitz', Total_moves='66', Black_blunders='4', White_blunders='2', Black_mistakes='0', White_mistakes='3', Black_inaccuracies='3', White_inaccuracies='1', Black_inferior_moves='7', White_inferior_moves='6', Black_ts_moves='8', White_ts_moves='8', Black_ts_blunders='0', White_ts_blunders='0', Black_ts_mistakes='0', White_ts_mistake='0', Black_long_moves='2', White_long_moves='1', Black_bad_long_moves='1', White_bad_long_moves='1', Game_flips='8', Game_flips_ts='0')

In [5]:
#Pregunta 1
df.count()

3739909

In [6]:
type(df.groupBy("Result"))

pyspark.sql.group.GroupedData

In [7]:
#Pregunta 2
df.groupBy("Result").count().collect()

[Row(Result='*', count=72),
 Row(Result='1/2-1/2', count=109548),
 Row(Result='1-0', count=1863894),
 Row(Result='0-1', count=1766395)]

In [8]:
#Pregunta 3
sorted(df.groupBy("Termination").count().collect(), key=lambda x: x["count"])[-1]

Row(Termination='Normal', count=2869458)

In [9]:
#Pregunta 4
df.filter((df["Blackelo"] > 1900) & (df["WhiteElo"] > 1900)).count()

618272

In [10]:
#Pregunta 5
sorted(df.filter((df["Blackelo"] > 1900) & (df["WhiteElo"] > 1900)).groupBy("Event").count().collect(), key=lambda x: x["count"])[-1]

Row(Event='Rated Blitz game', count=279935)

In [11]:
#Pregunta 6
spark.createDataFrame(df.filter((df["Blackelo"] > 1900) & (df["WhiteElo"] > 1900)).groupBy("Opening").count().collect()).select(["Opening"]).show()

+--------------------+
|             Opening|
+--------------------+
|Grünfeld Defense:...|
|King's Gambit, Fa...|
|Four Knights Game...|
|Alekhine Defense:...|
|King's Indian Att...|
|Ruy Lopez: Classi...|
|Blackmar-Diemer G...|
|Caro-Kann Defense...|
|Owen Defense: Mat...|
|Queen's Gambit Ac...|
|Modern Defense: R...|
|Queen's Gambit De...|
|Caro-Kann Defense...|
|Evans Gambit, Leo...|
|Slav Defense: Cze...|
|Ruy Lopez: Berlin...|
|Tarrasch Defense:...|
|Evans Gambit, Mor...|
|King's Indian Def...|
|French Defense: S...|
+--------------------+
only showing top 20 rows



In [12]:
#Pregunta 7
df.filter((df["Result"] == "0-1") & (df["Opening"].contains("Sicilian Defense"))).count()

250672

In [13]:
dfAperturas = spark.read.csv("high_elo_opening.csv", header=True)

In [14]:
dfAperturas.head()

Row(opening_name='Alekhine Defense, Balogh Variation', side='white', num_games='692', ECO='B03', last_played_date='2018-06-22', perf_rating='2247', avg_player='2225', perc_player_win='40.8', perc_draw='24.3', pec_opponent_win='35.0', moves_list="['1.e4', 'Nf6', '2.e5', 'Nd5', '3.d4', 'd6', '4.Bc4']", move1w='e4', move1b='Nf6', move2w='e5', move2b='Nd5', move3w='d4', move3b='d6', move4w='Bc4', move4b=None, perc_white_win='40.8', perc_black_win='35.0', white_odds='1.1657142857142857', white_wins='282.336', black_wins='242.2')

In [15]:
#Pregunta 8
df.filter(df["Opening"].contains("Sicilian Defense")).join(dfAperturas, df["ECO"] == dfAperturas["ECO"], "inner").select(["Opening", "num_games"]).show()

+--------------------+---------+
|             Opening|num_games|
+--------------------+---------+
|Sicilian Defense:...|      146|
|Sicilian Defense:...|      924|
|Sicilian Defense:...|    10310|
|Sicilian Defense:...|     1749|
|Sicilian Defense:...|     1791|
|Sicilian Defense:...|    22482|
|Sicilian Defense:...|    16093|
|Sicilian Defense:...|    17184|
|Sicilian Defense:...|      213|
|Sicilian Defense:...|      512|
|Sicilian Defense:...|     1096|
|Sicilian Defense:...|      254|
|Sicilian Defense:...|     1217|
|Sicilian Defense:...|      124|
|Sicilian Defense:...|      713|
|Sicilian Defense:...|      288|
|Sicilian Defense:...|      202|
|Sicilian Defense:...|      362|
|Sicilian Defense:...|     2274|
|Sicilian Defense:...|      219|
+--------------------+---------+
only showing top 20 rows



In [16]:
#Pregunta 9
df.filter((df["Blackelo"] < 1900) & (df["WhiteElo"] < 1900)).join(dfAperturas, df["ECO"] == dfAperturas["ECO"], "inner").select(["Opening", "num_games"]).show()

+-----------------+---------+
|          Opening|num_games|
+-----------------+---------+
|     Bird Opening|     6728|
|     Bird Opening|      493|
|     Bird Opening|      738|
|     Réti Opening|    15625|
|     Réti Opening|      463|
|     Réti Opening|    20045|
|     Réti Opening|      895|
|     Réti Opening|     1434|
|     Réti Opening|     3596|
|     Réti Opening|      572|
|     Réti Opening|     3309|
|     Réti Opening|     5579|
|     Réti Opening|      637|
|     Réti Opening|      306|
|     Réti Opening|     8899|
|     Réti Opening|     6773|
|     Réti Opening|     1035|
|     Réti Opening|      329|
|Queen's Pawn Game|     2199|
|Queen's Pawn Game|      462|
+-----------------+---------+
only showing top 20 rows

